# Pre Config


1.   importing librarys
2.   checking gpu
3.   mounting google drive





In [ ]:
# importing all librarys
from google.colab import drive
import tensorflow as tf
import os
from PIL import Image
import numpy as np
import PIL.ImageShow
import time
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Converting Data to JPEG
1. converting pdf to jpeg
2. split catagories to their directory
3. spliting to train, test, val

open_resize_and_label_png_directory

In [ ]:
# Initialize lists to store images and labels
images = []
labels = []

# Loop through the dataset directory
for root, dirs, files in os.walk('/content/drive/MyDrive/Articles_Seperation_DL/labeled_data'):
    for file in files:
        # Load the image using PIL
        image_path = os.path.join(root, file)
        try:
          image = Image.open(image_path)

          # Resize the image
          image = image.resize((224, 224))
          image = np.array(image)
          image = image.reshape((224, 224, 1))

          # Normalize the pixel values to [0, 1]
          image = np.array(image) / 255.0


          # Append the image and label to the respective lists
          images.append(image)

          # Extract the label from the directory name
          label = os.path.basename(root)
          labels.append(label)

        except:
          print(image_path, ' was not loaded succefully')

In [ ]:
# Convert the lists to NumPy arrays
X = np.array(images)
y = np.array(labels)
y = np.where(y == 'first_pages', 1, 0)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define the model
model = tf.keras.Sequential()

# Add a convolutional layer with 32 filters, a 3x3 kernel, and 'relu' activation
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(224, 224, 1)))

# Add a max pooling layer with 2x2 pool size
model.add(layers.MaxPooling2D((2, 2)))

# Add another convolutional layer with 64 filters and 'relu' activation
model.add(layers.Conv2D(64, (3, 3), activation='relu'))

# Flatten the output from the previous layer
model.add(layers.Flatten())

# Add a dense (fully connected) layer with 64 units and 'relu' activation
model.add(layers.Dense(32, activation='relu'))

# Add an output layer with 1 unit and 'sigmoid' activation for binary classification
model.add(layers.Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
# Define the number of epochs and batch size
epochs = 10
batch_size = 64

In [ ]:
# Train the model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=epochs, batch_size=batch_size)

# Evaluate the model on the test data
test_loss, test_accuracy = model.evaluate(X_test, y_test)
# Print out the model's accuracy
print('\nTest loss:', test_loss)
print('Test accuracy:', test_accuracy)

# Print the model summary
print(model.summary())

In [ ]:
# Save the model to h5 file
model.save('/content/drive/MyDrive/Articles_Seperation_DL/forth_model.h5')

In [ ]:
# Load the model
model = tf.keras.models.load_model('/content/drive/MyDrive/Articles_Seperation_DL/first_model.h5')

In [ ]:
y_test_pred = model.predict(X_test).reshape(-1)


In [ ]:
y_test_pred_classes = (y_test_pred > 0.5)

In [ ]:
mistakes = np.where(y_test != y_test_pred_classes)[0]

In [ ]:
mistakes_images = X_test[mistakes, :, :, :]

In [ ]:
print(mistakes_images.shape)

In [ ]:
tru_labels_of_mistakes = y_test[mistakes]

In [ ]:
print()
img = Image.fromarray(mistakes_images[1].reshape(224, 224), mode='L')
img.show()

In [ ]:
for i in range(len(mistakes)):
    img = Image.fromarray(mistakes_images[i].reshape(224, 224) * 255, mode='L')
    img.save('/content/drive/MyDrive/Articles_Seperation_DL/'
     + str(i) + 'true label' + str(tru_labels_of_mistakes[i]) + '.png')

In [ ]:
drive.flush_and_unmount()
print('All changes made in this colab session should now be visible in Drive.')